Use peaks defined in Xu et al K562 dataset to extract ATAC matrix

In [1]:
library(Seurat)
library(Signac)
library(magrittr)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(ggplot2)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDupl

Specify file path

In [2]:
dir.scE2G_res = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_K562_Xu/K562/"
path.obj.seurat = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/PBMC/1.prepare_data/1.seurat_pipeline.240620/obj.seurat.qc.56.filter.rds"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/PBMC/1.prepare_data/2.1.use_K562_Xu_peaks.240715/"

In [3]:
dir.create(dir.output,recursive = T)

Import seurat object

In [5]:
obj.seurat = readRDS(path.obj.seurat)

Import E-G pairs for Xu et al K562

In [6]:
pairs.E2G = readGeneric(paste(dir.scE2G_res,
                              "/",
                              "Kendall/Pairs.tsv.gz",sep = ""),
                        header = T,
                        keep.all.metadata = T)

Merge peaks

In [7]:
bed.peaks = reduce(pairs.E2G)
bed.peaks

GRanges object with 157600 ranges and 0 metadata columns:
           seqnames              ranges strand
              <Rle>           <IRanges>  <Rle>
       [1]     chr1          9926-10618      *
       [2]     chr1         15894-16509      *
       [3]     chr1         17271-17713      *
       [4]     chr1         29230-29495      *
       [5]     chr1       115493-115961      *
       ...      ...                 ...    ...
  [157596]     chrX 156009722-156010137      *
  [157597]     chrX 156016191-156016823      *
  [157598]     chrX 156019811-156020191      *
  [157599]     chrX 156025028-156025591      *
  [157600]     chrX 156030021-156030865      *
  -------
  seqinfo: 23 sequences from an unspecified genome; no seqlengths

Generate ATAC matrix

In [8]:
future::plan("multicore", workers = 32)
atac.matrix <- FeatureMatrix(
  fragments = obj.seurat@assays$ATAC_empty@fragments,
  features = bed.peaks,
  cells = colnames(obj.seurat@assays$RNA$counts)
)
future::plan("sequential")

Extracting reads overlapping genomic regions

Extracting reads overlapping genomic regions



Save data

In [9]:
saveRDS(atac.matrix,
        paste(dir.output, "atac.matrix.PBMC.rds", sep = "/"))

In [10]:
saveRDS(obj.seurat@assays$RNA$counts,
        paste(dir.output, "matrix.rna.PBMC.rds", sep = "/"))

In [11]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
[1] stats4    grid      stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] ggplot2_3.5.1        Matrix_1.6-5         GenomicRanges_1.54.1
 [4] GenomeInfoDb_1.38.1  IRanges_2.36.0       S4Vectors_0.40.2   